Modelo para anlisis de sentmientos para luego precedir si tiene sintomas de depresion.

In [1]:
from string import punctuation
import pandas as pd
import nltk
import re

In [2]:
# read the data from tweets_public.csv and create a dataframe
df = pd.read_csv('data/tweets_public.csv', encoding='utf-8')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,926419989107798016,NaN,Madrid
1,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,934854385577943041,NaN,Mexico City
2,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,945318406441635840,NaN,Madrid
3,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,927540721296568320,NaN,Pacific Time (US & Canada)
4,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,947965901332197376,NaN,Buenos Aires


In [3]:
df.shape

(7867, 10)

# Preprocesamiento
Limpieza de datos

In [4]:
# create a new dataframe with only the text and airline_sentiment columns and tweet id with the name df_sentiment
df_sentiment = df[['text', 'airline_sentiment', 'tweet_id']]

In [5]:
# transform the text letters to lowercase
df_sentiment['text'] = df_sentiment['text'].str.lower()
df_sentiment.head()

C:\Users\raula\AppData\Local\Temp\ipykernel_12380\3390815067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['text'] = df_sentiment['text'].str.lower()


,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma: https://t.co/r...,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [6]:
# Referencia: https://stackoverflow.com/questions/6718633/python-regular-expression-again-match-url
# remove the urls from the text but keep all the text after the url
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.split('http[s]*\S+', str(x))[0])
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma:,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [7]:
# remove the punctuation from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair\nb...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [8]:
# change the \n to a space
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: x.replace('\n', ' '))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair bu...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [9]:
# remove the stopwrods from the text
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df_sentiment.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [10]:
# remove the emojis from the text

# regular expression pattern to remove emojis from text
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # faces
        u"\U0001F300-\U0001F5FF"  # simbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)

# use lambda function to remove the emojis from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: emoji_pattern.sub(r'', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [11]:
# remove the numbers from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.sub(r'\d+', '', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


# BERT

In [12]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

c:\Users\raula\Documents\RAUL_ANGEL\UVG_COMPU\NOVENO_SEMESTRE\DISENO-E-INOVACION\analisis_sentimientos_depresion\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Inicialización
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
# DATASET_PATH = '/content/drive/My Drive/videos/2020-07-20/BERT_sentiment_IMDB_Dataset.csv'
NCLASSES = 3

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
# # Cargar dataset
# from google.colab import drive
# drive.mount('/content/drive')

# df_bert = pd.read_csv(DATASET_PATH)
# df_bert = df[0:10000]

df_bert = df_sentiment[['text', 'airline_sentiment']]

In [15]:
print(df.head())
print(df.shape)
print("\n".join(wrap(df['text'][200])))

  airline_sentiment  is_reply  reply_count  retweet_count   
0           neutral     False            0              0  \
1           neutral      True            0              0   
2          negative     False            0              0   
3          negative     False            0              0   
4          positive      True            0              0   

                                                text tweet_coord   
0  Trabajar en #Ryanair como #TMA: https://t.co/r...         NaN  \
1  @Iberia @FIONAFERRER Cuando gusten en Cancún s...         NaN   
2  Sabiais que @Iberia te trata muy bien en santi...         NaN   
3  NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...         NaN   
4  @cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...         NaN   

                    tweet_created            tweet_id tweet_location   
0  Fri Nov 03 12:05:12 +0000 2017  926419989107798016            NaN  \
1  Sun Nov 26 18:40:28 +0000 2017  934854385577943041            NaN   
2  Mon D

In [16]:
# Reajustar dataset
df_bert['label'] = df_bert['airline_sentiment'].apply(lambda x: 1 if x == 'positive' else 0 if x == 'neutral' else -1)
df_bert = df_bert.drop(['airline_sentiment'], axis=1)
df_bert.head()

,text,label
0,trabajar ryanair tma,0
1,iberia fionaferrer gusten cancún viaja disfrut...,0
2,sabiais iberia trata bien santiago chilete cam...,-1
3,nunca nunca nunca pidáis café ryanair bueno ve...,-1
4,cristortu dakar iberia mitsubishies bfgoodrich...,1


In [17]:
# TOKENIZACIÓN
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [18]:
# Ejemplo tokenización
sample_txt = 'Me gusto mucho esa pelicula!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Frase:  Me gusto mucho esa pelicula!
Tokens:  ['Me', 'gusto', 'mucho', 'esa', 'peli', '##cula', '!']
Tokens numéricos:  [11589, 67782, 23827, 16186, 100475, 32107, 106]


In [19]:
# Codificación para introducir a BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 10,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

c:\Users\raula\Documents\RAUL_ANGEL\UVG_COMPU\NOVENO_SEMESTRE\DISENO-E-INOVACION\analisis_sentimientos_depresion\venv\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [21]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'Me', 'gusto', 'mucho', 'esa', 'peli', '##cula', '!', '[SEP]', '[PAD]']
tensor([   101,  11589,  67782,  23827,  16186, 100475,  32107,    106,    102,
             0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])


In [22]:
# CREACIÓN DATASET

class TEXTDataset(Dataset):

  def __init__(self,texts,labels,tokenizer,max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.texts)
    
  def __getitem__(self, item):
    text = str(self.texts[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        text,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    

    return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      } 

In [23]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = TEXTDataset(
      texts = df.text.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [24]:
df_train_bert, df_test_bert = train_test_split(df_bert, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train_bert, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test_bert, tokenizer, MAX_LEN, BATCH_SIZE)

In [25]:
# Bert Model
class BERTSentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    return self.linear(drop_output)


In [26]:
model = BERTSentimentClassifier(NCLASSES)
model = model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# print(model)

In [28]:
# Training
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

c:\Users\raula\Documents\RAUL_ANGEL\UVG_COMPU\NOVENO_SEMESTRE\DISENO-E-INOVACION\analisis_sentimientos_depresion\venv\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [30]:
# Entrenamiento!!!

for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1} de {EPOCHS}')
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train_bert)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test_bert)
  )
  print(f'Entrenamiento: Loss: {train_loss}, accuracy: {train_acc}')
  print(f'Validación: Loss: {test_loss}, accuracy: {test_acc}')
  print('')

Epoch 1 de 5
------------------


In [ ]:
def classifySentiment(text):
  encoding_review = tokenizer.encode_plus(
      text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  print('predicción = ', prediction)
  print("\n".join(wrap(text)))

